### Load Data

In [41]:
import csv
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

images = []
measurements = []
with open('./data/driving_log.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        correction = 0.2
        
        c_steer = float(line[3])
        l_steer = c_steer + correction
        r_steer = c_steer - correction
        
        directory = './data/IMG/'
        c_file = line[0].split('/')[-1]
        c_img = cv2.cvtColor(cv2.imread(directory + c_file), cv2.COLOR_BGR2RGB)
        
        l_file = line[1].split('/')[-1]
        l_img = cv2.cvtColor(cv2.imread(directory + l_file), cv2.COLOR_BGR2RGB)
        
        r_file = line[2].split('/')[-1]
        r_img = cv2.cvtColor(cv2.imread(directory + r_file), cv2.COLOR_BGR2RGB)
        
        images.extend((c_img, l_img, r_img, cv2.flip(c_img, 1), cv2.flip(l_img, 1), cv2.flip(r_img, 1)))
        measurements.extend((c_steer, l_steer, r_steer, c_steer*-1.0, l_steer*-1.0, r_steer*-1.0))
    
X_train = np.array(images)
y_train = np.array(measurements)

print(X_train.shape)
print(y_train.shape)

(19662, 160, 320, 3)
(19662,)


### Build and Train Model

In [46]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Dropout, Cropping2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25),(0,0))))

model.add(Conv2D(6, (5, 5), activation="relu"))
model.add(MaxPooling2D())
model.add(Conv2D(6, (5, 5), activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(120))
model.add(Dropout(0.5))
model.add(Dense(84))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

model.save('model.h5')

Train on 15729 samples, validate on 3933 samples
Epoch 1/5
15729/15729 [==============================] - 42s - loss: 0.0366 - val_loss: 0.0276
Epoch 2/5
15729/15729 [==============================] - 41s - loss: 0.0203 - val_loss: 0.0245
Epoch 3/5
15729/15729 [==============================] - 41s - loss: 0.0174 - val_loss: 0.0242
Epoch 4/5
15729/15729 [==============================] - 41s - loss: 0.0161 - val_loss: 0.0232
Epoch 5/5
15729/15729 [==============================] - 41s - loss: 0.0153 - val_loss: 0.0236


In [47]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Dropout, Cropping2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20),(0,0))))

model.add(Conv2D(24, (5, 5), strides=(2, 2), activation="relu"))
model.add(Conv2D(36, (5, 5), strides=(2, 2), activation="relu"))
model.add(Conv2D(48, (5, 5), strides=(2, 2), activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100))
model.add(Dropout(0.5))
model.add(Dense(50))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

model.save('model1.h5')

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  # Remove the CWD from sys.path while we load stuff.
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  # This is added back by InteractiveShellApp.init_path()
/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
  if sys.path[0] == '':


Train on 15729 samples, validate on 3933 samples
Epoch 1/5
15729/15729 [==============================] - 53s - loss: 0.0242 - val_loss: 0.0273
Epoch 2/5
15729/15729 [==============================] - 52s - loss: 0.0171 - val_loss: 0.0253
Epoch 3/5
15729/15729 [==============================] - 52s - loss: 0.0148 - val_loss: 0.0291
Epoch 4/5
15729/15729 [==============================] - 52s - loss: 0.0135 - val_loss: 0.0270
Epoch 5/5
15729/15729 [==============================] - 52s - loss: 0.0123 - val_loss: 0.0271
